<a href="https://colab.research.google.com/github/salmanromeo/CS_5683_Big_Data_Analytics/blob/main/GCN_Bonus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Custom neighborhood aggregation: As you can notice, the GCN aggregates from all its neighbors, which can be costly at times. The idea for this task is to set a maximum number of random neighbors to aggregate information from. This can be done after our data pre-processing step by simply converting some entries of the adjacency matrix to zeroes. Evaluate this model's performance against all other models. Note: This has to be done for only 2-layer models and each layer should have different random samples for neighborhood aggregation.**

In [57]:
import numpy as np
import scipy.sparse as sp
import torch

In [58]:
def encode_onehot(labels):
    classes = set(labels)
    classes_dict = {c: np.identity(len(classes))[i, :] for i, c in enumerate(classes)}
    labels_onehot = np.array(list(map(classes_dict.get, labels)), dtype=np.int32)
    return labels_onehot

In [59]:
def feature_normalize(mx):
    """Row-normalize sparse matrix"""
    rowsum = np.array(mx.sum(1))
    r_inv = np.power(rowsum, -1).flatten()
    r_inv[np.isinf(r_inv)] = 0.
    r_mat_inv = sp.diags(r_inv)
    mx = r_mat_inv.dot(mx)
    return mx

In [60]:
def adj_normalize(mx):
    """Row-normalize sparse matrix"""
    rowsum = np.array(mx.sum(1)) # Sum each row
    r_inv = np.power(rowsum, -1/2).flatten() # Negative square root
    r_mat_inv = sp.diags(r_inv) # Create diagonal matrix
    mx = r_mat_inv.dot(mx)
    mx = mx.dot(r_mat_inv)
    return mx

In [61]:
def sparse_mx_to_torch_sparse_tensor(sparse_mx):
    """Convert a scipy sparse matrix to a torch sparse tensor."""
    sparse_mx = sparse_mx.tocoo().astype(np.float32)
    indices = torch.from_numpy(
        np.vstack((sparse_mx.row, sparse_mx.col)).astype(np.int64))
    values = torch.from_numpy(sparse_mx.data)
    shape = torch.Size(sparse_mx.shape)
    return torch.sparse.FloatTensor(indices, values, shape)

In [62]:
def load_data(path="/content/data/cora/", dataset="cora"):
    """Load citation network dataset (cora only for now)"""
    print('Loading {} dataset...'.format(dataset))

    idx_features_labels = np.genfromtxt("{}{}.content".format(path, dataset),
                                        dtype=np.dtype(str))
    features = sp.csr_matrix(idx_features_labels[:, 1:-1], dtype=np.float32)
    labels = encode_onehot(idx_features_labels[:, -1]) # one-hot encoding the labels

    idx = np.array(idx_features_labels[:, 0], dtype=np.int32)
    idx_map = {j: i for i, j in enumerate(idx)}

    edges_unordered = np.genfromtxt("{}{}.cites".format(path, dataset),
                                    dtype=np.int32)
    edges = np.array(list(map(idx_map.get, edges_unordered.flatten())),
                     dtype=np.int32).reshape(edges_unordered.shape)

    adj = sp.coo_matrix((np.ones(edges.shape[0]), (edges[:, 0], edges[:, 1])),
                        shape=(labels.shape[0], labels.shape[0]),
                        dtype=np.float32)

    features = feature_normalize(features)

    adj = adj_normalize(adj + sp.eye(adj.shape[0]))

    idx_train = range(140)
    idx_val = range(200, 500)
    idx_test = range(500, 1500)

    features = torch.FloatTensor(np.array(features.todense()))
    labels = torch.LongTensor(np.where(labels)[1])
    adj = sparse_mx_to_torch_sparse_tensor(adj)

    idx_train = torch.LongTensor(idx_train)
    idx_val = torch.LongTensor(idx_val)
    idx_test = torch.LongTensor(idx_test)

    return adj, features, labels, idx_train, idx_val, idx_test

In [63]:
def accuracy(output, labels):
    preds = output.max(1)[1].type_as(labels)
    correct = preds.eq(labels).double()
    correct = correct.sum()
    return correct / len(labels)

In [64]:
import math
from torch.nn.parameter import Parameter
from torch.nn.modules.module import Module

class GraphConvolution(Module):
    def __init__(self, in_feat, out_feat, bias=True):
        super(GraphConvolution, self).__init__()
        self.in_features = in_feat
        self.out_features = out_feat
        self.weight = Parameter(torch.FloatTensor(in_feat, out_feat))
        if bias:
            self.bias = Parameter(torch.FloatTensor(out_feat))
        else:
            self.register_parameter('bias', None)
        self.reset_parameters()

    def reset_parameters(self):
        stdv = 1. / math.sqrt(self.weight.size(1))
        self.weight.data.uniform_(-stdv, stdv)
        if self.bias is not None:
            self.bias.data.uniform_(-stdv, stdv)

    def forward(self, inp, adj, max_neighbors=20):
        # Apply random masking to the adjacency matrix
        num_nodes = adj.shape[0]
        mask = torch.rand(num_nodes, num_nodes).to(adj.device)
        mask = mask < (max_neighbors / num_nodes)  # Mask only a subset of neighbors

        # Apply the mask to the adjacency matrix
        adj_masked = adj.to_dense() * mask

        support = torch.mm(inp, self.weight)
        output = torch.spmm(adj_masked, support)  # Use masked adjacency matrix

        if self.bias is not None:
            return output + self.bias
        else:
            return output

In [71]:
import torch.nn as nn
import torch.nn.functional as F

class GCN(nn.Module):
    def __init__(self, nfeat, nhid, nclass, dropout):
        super(GCN, self).__init__()
        self.gcn1 = GraphConvolution(nfeat, nhid)
        self.gcnn2 = GraphConvolution(nhid, nhid)
        self.gcn2 = GraphConvolution(nhid, nclass)
        self.dropout = dropout

    def forward(self, x, adj, max_neighbors=20):
        x = F.relu(self.gcn1(x, adj, max_neighbors))  # Apply random neighborhood aggregation
        x = F.dropout(x, self.dropout, training=self.training)
        x = F.relu(self.gcnn2(x, adj, max_neighbors))  # Apply random neighborhood aggregation
        x = F.dropout(x, self.dropout, training=self.training)
        x = self.gcn2(x, adj, max_neighbors)  # Apply random neighborhood aggregation
        return F.log_softmax(x, dim=1)

In [72]:
import torch.optim as optim

adj, features, labels, train_ids, val_ids, test_ids = load_data()

model = GCN(nfeat=features.shape[1],
            nhid=16,
            nclass=labels.max().item() + 1,
            dropout=0.5)

optimizer = optim.Adam(model.parameters(),
                       lr=0.01, weight_decay=5e-4)

Loading cora dataset...


In [73]:
# If CUDA is available, move all data to GPU
if torch.cuda.is_available():
    model.cuda()
    features = features.cuda()
    adj = adj.cuda()
    labels = labels.cuda()
    train_ids.cuda()
    val_ids.cuda()
    test_ids.cuda()

In [74]:
import time

def train(epoch):
    t = time.time()

    model.train()
    optimizer.zero_grad()
    output = model(features, adj)
    loss_train = F.nll_loss(output[train_ids], labels[train_ids])
    acc_train = accuracy(output[train_ids], labels[train_ids])
    loss_train.backward()
    optimizer.step()

    model.eval()
    output = model(features, adj)
    loss_val = F.nll_loss(output[val_ids], labels[val_ids])
    acc_val = accuracy(output[val_ids], labels[val_ids])

    print('Epoch: {:04d}'.format(epoch+1),
          'loss_train: {:.4f}'.format(loss_train.item()),
          'acc_train: {:.4f}'.format(acc_train.item()),
          'loss_val: {:.4f}'.format(loss_val.item()),
          'acc_val: {:.4f}'.format(acc_val.item()),
          'time: {:.4f}s'.format(time.time() - t))

In [75]:
from sklearn.metrics import precision_score, recall_score, f1_score

def test():
    model.eval()
    output = model(features, adj)
    loss_test = F.nll_loss(output[test_ids], labels[test_ids])
    acc_test = accuracy(output[test_ids], labels[test_ids])
    _, predicted = torch.max(output[test_ids], 1)

    precision = precision_score(labels[test_ids].cpu(), predicted.cpu(), average='macro', zero_division=1)
    recall = recall_score(labels[test_ids].cpu(), predicted.cpu(), average='macro', zero_division=1)
    f1 = f1_score(labels[test_ids].cpu(), predicted.cpu(), average='macro', zero_division=1)

    print("Test set results:",
          "loss= {:.4f}".format(loss_test.item()),
          "accuracy= {:.4f}".format(acc_test.item()),
          "precision= {:.4f}".format(precision),
          "recall= {:.4f}".format(recall),
          "F1= {:.4f}".format(f1))

In [76]:
t_total = time.time()

for epoch in range(200):
    train(epoch)

print('Optimization finished...')
print('Total time elapsed: {:.4f}s'.format(time.time() - t_total))

test()

Epoch: 0001 loss_train: 1.9739 acc_train: 0.0786 loss_val: 1.9823 acc_val: 0.0833 time: 1.3068s
Epoch: 0002 loss_train: 1.9699 acc_train: 0.0857 loss_val: 1.9775 acc_val: 0.0833 time: 0.9172s
Epoch: 0003 loss_train: 1.9682 acc_train: 0.0714 loss_val: 1.9738 acc_val: 0.1567 time: 0.8363s
Epoch: 0004 loss_train: 1.9600 acc_train: 0.2000 loss_val: 1.9695 acc_val: 0.1567 time: 0.6142s
Epoch: 0005 loss_train: 1.9578 acc_train: 0.1929 loss_val: 1.9658 acc_val: 0.1567 time: 0.6202s
Epoch: 0006 loss_train: 1.9536 acc_train: 0.2000 loss_val: 1.9619 acc_val: 0.1567 time: 0.6149s
Epoch: 0007 loss_train: 1.9492 acc_train: 0.2000 loss_val: 1.9581 acc_val: 0.1567 time: 0.6464s
Epoch: 0008 loss_train: 1.9461 acc_train: 0.2000 loss_val: 1.9539 acc_val: 0.1567 time: 0.6214s
Epoch: 0009 loss_train: 1.9417 acc_train: 0.2000 loss_val: 1.9499 acc_val: 0.1567 time: 0.6444s
Epoch: 0010 loss_train: 1.9392 acc_train: 0.2000 loss_val: 1.9469 acc_val: 0.1567 time: 0.6290s
Epoch: 0011 loss_train: 1.9336 acc_train

###